# Code Writing

In [23]:
import os
from dotenv import load_dotenv

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [24]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import (ChatPromptTemplate)
from langchain_experimental.utilities import PythonREPL
from langchain_openai import ChatOpenAI

In [25]:
template = """Write some python code to solve the user's problem. 

Return only python code in Markdown format, e.g.:

```python
....
```"""
prompt = ChatPromptTemplate.from_messages([("system", template), ("human", "{input}")])

model = ChatOpenAI()

In [26]:
def _sanitize_output(text: str):
    _, after = text.split("```python")
    return after.split("```")[0]

In [27]:
chain = prompt | model | StrOutputParser() | _sanitize_output | PythonREPL().run

In [36]:
chain.invoke({"input": "Give me a list of fibonacci numbers"})

'[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181]\n'

## Agent designed to write and execute Python code

In [29]:
from langchain import hub
from langchain.agents import AgentExecutor
from langchain_experimental.tools import PythonREPLTool

In [30]:
tools = [PythonREPLTool()]

In [31]:
from langchain.agents import create_openai_functions_agent
from langchain_openai import ChatOpenAI

In [32]:
instructions = """You are an agent designed to write and execute python code to answer questions.
You have access to a python REPL, which you can use to execute python code.
If you get an error, debug your code and try again.
Only use the output of your code to answer the question. 
You might know the answer without running any code, but you should still run the code to get the answer.
If it does not seem like you can write code to answer the question, just return "I don't know" as the answer.
"""
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)

In [33]:
agent = create_openai_functions_agent(ChatOpenAI(temperature=0), tools, prompt)

In [34]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [35]:
agent_executor.invoke({"input": "What is the 10th fibonacci number?"})



> Entering new AgentExecutor chain...

Invoking: `Python_REPL` with `def fibonacci(n):
    if n <= 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)

fibonacci(10)`


The 10th Fibonacci number is 55.

> Finished chain.


{'input': 'What is the 10th fibonacci number?',
 'output': 'The 10th Fibonacci number is 55.'}